In [29]:
import pandas as pd
import requests
import gmaps
from config import gkey 

gmaps.configure(api_key=gkey)

# Study data files
arsenic = "Resources/arsenic.csv"
algae = "Resources/algae.csv"
metals = "Resources/quarterlyMetals.csv"

# Read the mouse data and the study results
arsenic = pd.read_csv(arsenic)
algae = pd.read_csv(algae)
metals = pd.read_csv(metals)

arsenic

,Site Number,Site location,Cluster,Site Acronym,Arsenic (ug/L),Perchlorate (ug/L),date
0,Green In,Greenway WTP Inlet,srp,Green In,8.1,1.272,2004-04-13
1,Green Out,Greenway WTP Outlet,srp,Green Out,8.4,0.990,2004-04-13
2,Havasu 1,Havasu lake Sample,NaN,Havasu 1,3.3,NaN,2004-04-13
3,Havasu 2,Havasu lake Sample,NaN,Havasu 2,2.9,NaN,2004-04-13
4,HTC,Head of the Tempe Canal,tempe,head of tempe canal,NaN,NaN,2004-04-13
...,...,...,...,...,...,...,...
3423,SOCA,South Canal below CAP Cross-conect,srp,South canal below CAP,7.5,NaN,2011-04-04
3424,SPT In,Tempe Canal - Inlet to Tempe's South Plant,tempe,SPT In,NaN,NaN,2011-04-04
3425,SPT Out,Tempe's South Plant treated water,tempe,SPT Out,NaN,NaN,2011-04-04
3426,UH In,Union Hills Inlet,cap,Union Hills Inlet,NaN,NaN,2011-04-04


In [50]:
arsenic_heat = arsenic.loc[arsenic["date"] == "2004-04-13"]
arsenic_heat = arsenic_heat[["Site location", "Arsenic (ug/L)"]]

In [59]:
arsenic_heat

,Site location,Arsenic (ug/L)
0,Greenway WTP Inlet,8.1
1,Greenway WTP Outlet,8.4
2,Havasu lake Sample,3.3
3,Havasu lake Sample,2.9
4,Head of the Tempe Canal,NaN
5,AZ Canal at Highway 87,11.1
6,Cross-Cut Canal - Inlet to Tempe's North Plant,9.5
7,Tempe's North Plant treated water,7.5
8,AZ Canal at Pima Rd.,10.0
9,Salt River below Saguaro (Blue Point Bridge),0.0


In [60]:
target_coordinates = "33.448376, -112.074036"
keyword = "Union Hills Inlet"
radius = 1000000

params = {
    "location": target_coordinates,
    "keyword": keyword,
    "radius": radius,
    "type": "water",
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#srp = srp.loc(srp["Site location"].unique()]
salt_locations = pd.DataFrame(columns = ["Name", "Lat", "Lng"])
for site in arsenic["Site location"].unique():
    try:
        params["keyword"] = site
        response_json = requests.get(base_url, params=params).json()
        location = {"Name": site, "Lat": response_json["results"][0]["geometry"]["location"]["lat"],
                   "Lng": response_json["results"][0]["geometry"]["location"]["lng"]}
        salt_locations = salt_locations.append(location,ignore_index=True)
        print(f"Adding {site}")
    except:
        print(f"No location data found for {site}")


No location data found for Greenway WTP Inlet
No location data found for Greenway WTP Outlet
No location data found for Havasu lake Sample
Adding Head of the Tempe Canal
Adding AZ Canal at Highway 87
No location data found for Cross-Cut Canal - Inlet to Tempe's North Plant
Adding Tempe's North Plant treated water
Adding AZ Canal at Pima Rd.
Adding Salt River below Saguaro (Blue Point Bridge)
No location data found for CAP Canal at Cross-connect
No location data found for AZ Canal above CAP Cross-connect
No location data found for AZ Canal below CAP Cross-connect
No location data found for AZ Canal - Inlet to Squaw Peak WTP
Adding Squaw Peak WTP treated water
No location data found for AZ Canal - Inlet to Deer Valley WTP
Adding Deer Valley WTP treated water
Adding South Canal at Val Vista WTP
Adding Val Vista WTP treated water
No location data found for Verde River at Tangle
Adding AZ Canal at Central Av.
No location data found for AZ Canal - 56th St. (Alma School Road)
Adding verde riv

In [62]:
salt_locations.to_csv("output.csv", index=True)

In [66]:
salt_locations = salt_locations.rename(columns = {"Name": "Site location"})
salt_locations_heat = arsenic_heat.merge(salt_locations, on="Site location", how="outer")
salt_locations_heat = salt_locations_heat.dropna()
coordinates = salt_locations_heat[["Lat","Lng"]]
salt_locations_heat

,Site location,Arsenic (ug/L),Lat,Lng
5,AZ Canal at Highway 87,11.1,33.534129,-111.915248
7,Tempe's North Plant treated water,7.5,33.445891,-111.936720
8,AZ Canal at Pima Rd.,10.0,33.531536,-111.891513
9,Salt River below Saguaro (Blue Point Bridge),0.0,33.555259,-111.573645
14,Squaw Peak WTP treated water,6.0,33.528661,-112.032240
16,Deer Valley WTP treated water,10.6,33.567909,-112.125169
20,AZ Canal at Central Av.,9.4,33.562697,-112.073894
22,verde river at beeline highway,5.6,33.621894,-111.567875
26,Bartlett Reservoir near outlet,14.6,33.873618,-112.149609
27,Bartlett Reservoir near outlet,16.0,33.873618,-112.149609


In [76]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
rating = salt_locations_heat["Arsenic (ug/L)"].astype(float)
heat_layer = gmaps.heatmap_layer(coordinates, weights=rating, 
                                 dissipating=False, max_intensity=200,
                                 point_radius=1)

fig = gmaps.figure(layout=figure_layout, map_type="HYBRID")  # Assign the marker layer to a variable
#markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
#fig.add_layer(markers)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…